In [1]:
import json
import pandas as pd
import numpy as np
import re
import json

In [2]:
# read in the original json file
path = "../data/raw/property.json"
with open(path, "r") as f:
    property_data = json.load(f)

In [3]:
property_df = pd.DataFrame()

rent = []
address =[]
features = []
type = []
furnitured = [] 
pool = []
gym = []
latitude = []
longitude = []

# store each feature in the dataframe
for i in property_data:
    address.append(property_data[i]["name"])
    rent.append(property_data[i]["cost_text"])
    features. append(property_data[i]["features"])
    type.append(property_data[i]["type"])
    furnitured.append(property_data[i]["furnitured"])
    pool.append(property_data[i]["pool"])
    gym.append(property_data[i]["gym"])
    latitude.append(property_data[i]["coordinates"][0])
    longitude.append(property_data[i]["coordinates"][1])


property_df["address"] = address
property_df["rent"] = rent
property_df["features"] = features
property_df["type"] = type
property_df["furnitured"] = furnitured
property_df["pool"] = pool
property_df["gym"] = gym
property_df["latitude"] = latitude
property_df["longitude"] = longitude


,address,rent,features,type,furnitured,pool,gym,latitude,longitude
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,$400 Per Week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810283,144.956669
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,$350 per week,1 Bed1 Bath− Parking,Studio,Yes,No,No,-37.810779,144.968551
2,911/408 Lonsdale Street Melbourne VIC 3000,$330 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812598,144.960401
3,918/422 Collins St Melbourne VIC 3000,$600 Per week fully furnished,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.817097,144.960149
4,602/118 Franklin Street Melbourne VIC 3000,$330,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.808205,144.958903


In [4]:
property_df.shape

(15117, 9)

In [5]:
# check and drop duplicate
unique_property_df = property_df.drop_duplicates(keep="first").reset_index(drop = True)
unique_property_df.shape

(15004, 9)

In [6]:
# get postcode for all property
postcode=[]
for i in range(len(unique_property_df)):
    postcode.append(int(unique_property_df["address"][i][-4:]))

print("The number of postcode is: ", len(postcode))
unique_property_df["postcode"] = postcode

The number of postcode is:  15004


In [7]:
unique_property_df.shape

(15004, 10)

In [10]:
# drop the property with empty features
clean_features = unique_property_df[unique_property_df["features"] != ""].reset_index(drop=True)
clean_features.shape

(14970, 10)

In [12]:
num_bed_list = []
num_bath_list = []
num_park_list = []

# split the detail from the property features
for i in clean_features["features"]:

    # find the number of bedroom, bathroom and parking area
    num_bed = re.findall(r'(\d|-|−)\s?Bed', i)
    num_bath = re.findall(r'(\d|-|−)\s?Bath', i)
    num_park = re.findall(r'(\d|-|−)\s?Park', i)

    # if no information, set the number to zero
    if (len(num_bed) == 0) or (num_bed[0].isdigit() == False):
        num_bed = ["0"]
    if (len(num_bath) == 0) or (num_bath[0].isdigit() == False):
        num_bath = ["0"]
    if (len(num_park) == 0) or (num_park[0].isdigit() == False):
        num_park = ["0"]
    
    # append them together
    num_bed_list.append(int(num_bed[0]))
    num_bath_list.append(int(num_bath[0]))
    num_park_list.append(int(num_park[0]))

# generate the new features of the property
clean_features["num_bed"] = num_bed_list
clean_features["num_bath"] = num_bath_list
clean_features["num_park"] = num_park_list

In [14]:
# drop the property with no bedroom
clean_features = clean_features[clean_features["num_bed"] != 0].reset_index(drop=True)
clean_features.head()

,address,rent,features,type,furnitured,pool,gym,latitude,longitude,postcode,num_bed,num_bath,num_park
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,$400 Per Week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810283,144.956669,3000,1,1,0
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,$350 per week,1 Bed1 Bath− Parking,Studio,Yes,No,No,-37.810779,144.968551,3000,1,1,0
2,911/408 Lonsdale Street Melbourne VIC 3000,$330 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812598,144.960401,3000,1,1,0
3,918/422 Collins St Melbourne VIC 3000,$600 Per week fully furnished,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.817097,144.960149,3000,2,1,0
4,602/118 Franklin Street Melbourne VIC 3000,$330,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.808205,144.958903,3000,1,1,0


In [15]:
clean_features.shape

(14819, 13)

In [16]:
# save the cleaned property data with new features
clean_features.to_csv("../data/curated/cleaned_property_with_features.csv", index = None)